In [ ]:
import pandas as pd
from tqdm import tqdm

microdados = pd.DataFrame()

urls = [ ]
for year in range( 2020, 2020 + 1 ):
	for month in range( 1, 12 + 1 ):
		month = str( month ) if month >= 10 else "0" + str( month )
		url = f"https://www.gov.br/anac/pt-br/assuntos/regulados/empresas-aereas/envio-de-informacoes/microdados/basica{year}-{month}.zip"
		urls.append( url )

for url in tqdm( urls ):
	parcial = pd.read_csv( url, encoding = "latin1", sep = ";" ).sample( frac = 0.05, random_state = 2 )
	microdados = pd.concat( [ microdados, parcial ], axis = 0 )

url_est = 'https://www.gov.br/anac/pt-br/assuntos/dados-e-estatisticas/dados-estatisticos/arquivos/resumo_anual_2010.csv'


In [ ]:
# Da definição:

# Etapa Básica (flight stage): As etapas básicas são aquelas realizadas pela aeronave desde a sua decolagem até o próximo pouso, independentemente de onde tenha ocorrido o embarque ou o desembarque do objeto de transporte.
#
# Os dados estatísticos das etapas básicas representam o status da aeronave em cada etapa do voo, apresentando a movimentação de cargas e passageiros entre os aeródromos de origem e destino da aeronave. É a operação de uma aeronave entre um a decolagem e o próximo pouso, a ligação entre dois aeródromos.

# Etapa Combinada (On flight origin and destination - OFOD): As etapas combinadas identificam os pares de origem, onde ocorreu o embarque do objeto de transporte, e destino, onde ocorreu o desembarque do objeto de transporte, independentemente da existência de aeródromos intermediários, atendidos por determinado voo.

# É a etapa de voo vista com foco no objeto de transporte (pessoas e/ou cargas), com base no embarque e no desembarque nos aeródromos relacionados. Os dados estatísticos da etapa combinada informam a origem e destino dos passageiros e cargas transportadas no voo, independentemente das escalas realizadas.


#Aqui olharemos primeiramente para a etapa basica.

In [ ]:
microdados.duplicated( [ "id_basica" ] ).sum()
# de Fato, id basica é uma pk ( identificador unico ) nessa tabela, e não temos nenhum registro duplicado!

In [ ]:
(microdados.isna().sum( axis = 0 ) / microdados.shape[ 0 ]).sort_values( ascending = False )

# nr escala destino é 100% nulo. Doideira ( Isso provavelmente é algum tipo de informação que foi dropada ao longo do tempo. )

In [ ]:

unique = microdados.nm_empresa.value_counts()
cumulative = unique.cumsum()
cumulative = cumulative / unique.sum()
cumulative.sort_values( ascending = True ).iloc[ :15 ].plot( kind = "bar" )

# Nenhuma surpresa aqui: a gigantesca maioria das etapas é realizada por poucas empresas
# Aprox. 80% das etapas é feita por apenas 3;
#

In [ ]:
microdados.groupby( [ "nr_mes_referencia", "nm_mes_referencia" ] ).count()[ "id_basica" ].sort_index( ascending = True ).plot( kind = "bar" )

In [ ]:
microdados[ "nr_dia_semana_referencia" ] = pd.to_datetime( microdados[ "dt_referencia" ], infer_datetime_format = True ).dt.dayofweek

microdados.groupby( [ "nr_dia_semana_referencia", "nm_dia_semana_referencia" ] ).count()[ "id_basica" ].sort_index( ascending = True ).plot( kind = "bar" )

# Aqui temos uma surpresa: Fim de semanas são dias com menos voos? Interessante. [quinta, sexta, segunda, quarta, terca ] são dias de ocupação bem proxima.

In [ ]:
microdados.query( "nm_pais != 'BRASIL'" ).groupby( "nm_pais" ).nunique()[ "nm_empresa" ].nlargest( 10 ).plot( kind = "bar" )
# Essas são as nacionalidades das empresas não brasileiras dentro do dataset. USA domina, seguido de argentina, chile, venezuela

In [ ]:
microdados.groupby( [ "nm_municipio_destino", "nm_municipio_origem", "nm_empresa" ] ).nunique()

In [ ]:
microdados.value_counts( "nr_voo" )
# O numero de voo mais comum é 2317 , seguido de 2316, seguido de 248, 15 e 247
# Os menos comuns tem apenas 1 e são : 2263 , 6565, 595, 6566 , 593
# Atente-se à amostragem

In [ ]:
microdados.query( "nr_voo == 2317" )[ [ "nm_empresa", "nr_voo", "nr_singular", "id_di", "nm_municipio_destino", "nm_municipio_origem", "dt_referencia" ] ]
# O que identifica uma rota ( ORIGEM - DESTINO )  de maneira unica?!

In [ ]:
microdados["dt_referencia"] = pd.to_datetime(microdados.dt_referencia , infer_datetime_format = True)

In [ ]:
microdados.groupby( [ "nr_ano_referencia", "nr_mes_referencia" ]).mean()[["nr_ask",
																		  "nr_rpk",
																		  "nr_atk",
																		  "nr_rtk",
																		  "kg_peso"]].sort_index( ascending = True ).plot(  )